In [ ]:
import pandas as pd
import numpy as np
import os
import time
import math
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors


In [ ]:
bigdata = pd.read_csv("house_and_non_trad.csv")
address = bigdata["ADDRESS"]
bigdata.drop(["Unnamed: 0","SALE TYPE", "DAYS ON MARKET", "LOCATION", "ZIP OR POSTAL CODE", "SOLD DATE", "PROPERTY TYPE", "ADDRESS", "CITY", "STATE OR PROVINCE", "LOT SIZE", "STATUS" ,"LATITUDE", "LONGITUDE",
              'SOLD DATE', 'SALE TYPE', 'DAYS ON MARKET', 'LOT SIZE','STATE OR PROVINCE','LOCATION','HOA/MONTH', 'PROPERTY TYPE','CITY', 'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED',
'STATUS', 'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME', 'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)'], axis=1, inplace=True)
bigdata = bigdata.apply(pd.to_numeric)
bigdata["ID"] = bigdata.index
bigdata.index = address
bigdata

,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,ClosestHospital,ClosestGrocery,ClosestParks,ClosestWhole,ID
ADDRESS,,,,,,,,,,,
2204 Clifton Ave,41000,NaN,NaN,NaN,1920.0,NaN,3.363851,2.278936,1.582376,3.338943,0
1520 N Broadway,76000,NaN,1.0,1072.0,1915.0,71.0,1.214597,2.067144,1.519967,1.353327,1
936 Bennett Pl,76500,NaN,NaN,NaN,1920.0,NaN,1.078687,0.897029,0.939819,3.202665,2
2330 Madison Ave,169000,0.0,NaN,2682.0,1902.0,63.0,2.107550,1.254901,0.547803,2.683122,3
1128 Mckean Ave,9000,0.0,NaN,NaN,1920.0,NaN,2.449787,1.484299,1.925392,3.790267,4
...,...,...,...,...,...,...,...,...,...,...,...
120-A Midhurst Rd,1050000,5.0,5.5,4198.0,2017.0,250.0,2.072651,0.625980,2.351969,3.155345,4918
3037 E Baltimore St,360000,NaN,3.0,2400.0,2016.0,150.0,1.519639,2.419289,0.712085,2.419289,4919
2301 Hillhouse Rd,377000,NaN,3.5,2364.0,2015.0,159.0,5.450821,6.728278,6.049346,6.983671,4920


In [ ]:
class RecommenderAlgorithm():
  def __init__(self, bigdata, n_recommendations = 5):
    bigdata["ID"] = bigdata.index
    bigdata = bigdata.dropna()
    bigdata = bigdata.reset_index(drop=True)
    self.bigdata = bigdata
    self.model_knn = NearestNeighbors(metric='euclidean', algorithm='brute', n_neighbors=20, n_jobs=-1)
    self.n_recommendations = n_recommendations
  
  def make_recommendation(self, saved_prop):
    temp = self.bigdata.drop(["ID"], axis=1, inplace=False)
    normalized_df=(temp-temp.mean())/temp.std()
    normalized_df["PRICE"] = normalized_df["PRICE"] * 10
    normalized_df["BEDS"] = normalized_df["BEDS"] * 10
    normalized_df["BATHS"] = normalized_df["BATHS"] * 10
    normalized_df["SQUARE FEET"] = normalized_df["SQUARE FEET"] * 10
    # fit
    print(normalized_df)
    self.model_knn.fit(normalized_df)
    property_dict = {}
    for idx in saved_prop:
      distances, indices = self.model_knn.kneighbors(normalized_df.iloc[[idx]], n_neighbors=self.n_recommendations+1)
      # get list of raw idx of recommendations
      raw_recommends = \
          sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
      house_index_list = []
      for i in range(len(raw_recommends)):
        prop_id = raw_recommends[i][0]
        house_index_list.append(prop_id)
        if prop_id in property_dict.keys():
          property_dict[prop_id] = property_dict[prop_id] + i + 1
        else:
          property_dict[prop_id] = i + 1
    property_dict = dict(sorted(property_dict.items(), key = lambda x: x[1], reverse = True))
    print(property_dict)
    for i in range(len(saved_prop)):
      if saved_prop[i] in property_dict.keys():
        property_dict.pop(saved_prop[i])
    return property_dict

  def createRecDf(self, property_dict, my_favorite):
    first_rec = list(property_dict.keys())
    for i in range(len(my_favorite)):
      first_rec.insert(0, my_favorite[i])
      print(my_favorite[i])
    df = self.bigdata.iloc[first_rec]
    df = df[0:self.n_recommendations]
    return df




In [ ]:
saved = [1000]

In [ ]:
rec = RecommenderAlgorithm(bigdata,n_recommendations =  10)
recdict = rec.make_recommendation(saved)
recdf = rec.createRecDf(recdict,saved)
recdf

          PRICE       BEDS      BATHS  SQUARE FEET  YEAR BUILT  $/SQUARE FEET  \
0      6.127869 -26.453591 -14.698291    35.680818   -1.426978      -0.221112   
1     -7.150474 -26.453591 -14.698291     5.901415   -1.426978      -0.340603   
2     -7.514264 -26.453591  -4.098971    -7.121239   -1.131934      -0.218457   
3      2.126177 -26.453591  -9.398631     2.354026   -1.587912       0.017869   
4      2.853757 -26.453591  -4.098971    32.016738   -1.426978      -0.255632   
...         ...        ...        ...          ...         ...            ...   
2608  14.404097  18.801898  22.399327    32.355141    1.040668      -0.072413   
2609  -1.147935  18.801898   6.500348     6.834939    1.255246      -0.162695   
2610  15.950206  18.801898  11.800008    35.190719    1.067490      -0.072413   
2611  26.136332  18.801898  32.998647    27.815882    1.174779       0.169224   
2612   5.945974  18.801898  22.399327    11.502557    1.174779      -0.024617   

      ClosestHospital  Clos

,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,ClosestHospital,ClosestGrocery,ClosestParks,ClosestWhole,ID
1000,180000,3.0,1.5,1508.0,1952.0,119.0,2.200457,2.588708,4.884719,3.065919,6637 Loch Hl
1025,184000,3.0,1.5,1600.0,1959.0,115.0,2.696760,2.420611,5.092014,2.431279,1215 Brixton Rd
1138,169950,3.0,1.5,1500.0,1976.0,113.0,2.956283,2.689161,5.189379,5.681635,6118 Fairwood Ave
1300,210000,3.0,1.5,1515.0,1962.0,139.0,2.655744,0.935553,3.406113,4.031322,4811 Crosswood Ave
1237,175000,3.0,1.5,1534.0,1963.0,114.0,1.921250,2.113540,5.668859,6.799430,5662 Whitby Rd
926,175000,3.0,1.5,1567.0,1948.0,112.0,0.499259,2.398895,3.320647,3.270021,1545 Sherwood Ave
1225,182000,3.0,1.5,1440.0,1960.0,126.0,1.529850,1.692677,5.246077,6.443667,6013 Arizona Ave
1287,192000,3.0,1.5,1522.0,1965.0,126.0,2.107998,2.325605,5.929425,7.058503,5749 Utrecht Rd
1304,162000,3.0,1.5,1440.0,1960.0,113.0,1.505420,1.648352,5.158057,6.344946,5923 Arizona Ave
1125,190000,3.0,1.5,1437.0,1964.0,132.0,1.823873,2.062494,5.761893,6.985454,5673 Leiden Rd
